In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertConfig, BertForSequenceClassification, BertTokenizer, BertConfig
from datasets import load_dataset,load_metric
import numpy as np


from accelerate import Accelerator


accelerator = Accelerator()
device = accelerator.device

/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [2]:
# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
# texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
texts = "I don't like this movie."

# pretrained_name = "distilbert-base-uncased-finetuned-sst-2-english"
pretrained_name = "textattack/bert-base-uncased-SST-2"

# pred_config = DistilBertConfig.from_pretrained(pretrained_name)
# pred_tokenizer = DistilBertTokenizer.from_pretrained(pretrained_name)
# pred_model = DistilBertForSequenceClassification.from_pretrained(pretrained_name).to(device)

pred_config = BertConfig.from_pretrained(pretrained_name)
pred_tokenizer = BertTokenizer.from_pretrained(pretrained_name)
pred_model = BertForSequenceClassification.from_pretrained(pretrained_name).to(device)

inputs = pred_tokenizer(texts, return_tensors="pt")
with torch.no_grad():
    inputs = {key:val.to(device) for key,val in inputs.items()}
    logits = pred_model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(pred_model.config.id2label[predicted_class_id])

print(inputs)

NEGATIVE
{'input_ids': tensor([[ 101, 1045, 2123, 1005, 1056, 2066, 2023, 3185, 1012,  102]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [52]:
from maskgen.text_models.text_maskgen_model import MaskGeneratingModel
pred_hidden_dim = pred_model.config.dim
num_labels = pred_model.config.num_labels

mask_gen_model = MaskGeneratingModel(pred_model, hidden_size=pred_hidden_dim, num_classes=num_labels)
mask_gen_model.to(device)

mask_gen_model.load_state_dict(torch.load('text_mask_gen_model/mask_gen_model_0_30.pth'))
# mask_gen_model.load_state_dict(torch.load('trained/mask_gen_model12/mask_gen_model_2_90.pth'))
mask_gen_model.eval()
print()

In [57]:
# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
# texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
# texts = "I don't like this movie."
texts = 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have to always say "Gene Roddenberry\'s Earth..." otherwise people would not continue watching. Roddenberry\'s ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler. So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.'

# label = torch.tensor([1]).to(device)
label = None

inputs = pred_tokenizer(texts, return_tensors="pt")
with torch.no_grad():
    inputs = {key:val.to(device) for key,val in inputs.items()}
    logits = pred_model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(pred_model.config.id2label[predicted_class_id])


from captum.attr import visualization
expl = mask_gen_model.attribute_text(inputs['input_ids'], inputs['attention_mask'], label)[0][1:-1]
print(expl)
tokens = pred_tokenizer.convert_ids_to_tokens(inputs['input_ids'].flatten())[1:-1]
# normalize expl
expl = (expl - expl.min()) / (expl.max() - expl.min())
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                0,
                                0,
                                0,
                                0,
                                1,       
                                tokens,
                                1)]
                            
visualization.visualize_text(vis_data_records)

NEGATIVE
tensor([0.3210, 0.3563, 0.3453, 0.3491, 0.3525, 0.3653, 0.3398, 0.3578, 0.3543,
        0.3544, 0.3450, 0.3573, 0.3609, 0.3564, 0.3241, 0.3387, 0.3375, 0.3481,
        0.3336, 0.3385, 0.3355, 0.3249, 0.3221, 0.3223, 0.3267, 0.3246, 0.3104,
        0.3204, 0.3155, 0.3388, 0.3169, 0.3297, 0.3223, 0.3288, 0.3278, 0.3368,
        0.3356, 0.3328, 0.3419, 0.3345, 0.3410, 0.3435, 0.3297, 0.3284, 0.3214,
        0.3096, 0.3134, 0.3292, 0.3312, 0.3335, 0.3179, 0.3288, 0.3171, 0.3287,
        0.3266, 0.3224, 0.3227, 0.3517, 0.3411, 0.3226, 0.3154, 0.3364, 0.3289,
        0.3078, 0.2910, 0.3006, 0.3051, 0.2977, 0.2890, 0.2883, 0.2910, 0.2964,
        0.3059, 0.2882, 0.2954, 0.2912, 0.3257, 0.2911, 0.2928, 0.2940, 0.2897,
        0.2888, 0.3215, 0.2897, 0.3396, 0.3104, 0.3354, 0.3230, 0.2896, 0.2869,
        0.2881, 0.2864, 0.3064, 0.3021, 0.2866, 0.3099, 0.3148, 0.2959, 0.2942,
        0.3066, 0.3171, 0.3019, 0.3165, 0.3011, 0.2950, 0.3200, 0.3066, 0.2979,
        0.3243, 0.3002, 0.3028,

In [54]:
expl.shape

torch.Size([23])

In [51]:
len(tokens)

23